Conducts aperture photometry of cal star images. 

Before running:
- Have lists of individual cal star cubes
- Reduce those cal star cubes

Does the following:
- Attaches airmass
- Conducts aperture photometry by fitting Gaussian to the star in each image

In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from numpy import pi, r_
from scipy import optimize
import math

In [ ]:
#Use object-specific lists.

def airmass_appender(directory,image_list,airmasses):
    imlist = np.loadtxt(image_list,dtype=str)
    airmass_list = np.loadtxt(airmasses,dtype=str)

    for i in range(0,len(imlist)):

        im = fits.open(directory+imlist[i])
        header = im[0].header

        im_UT = header['UT']
        for j in range(0,len(airmass_list)):
            if str(airmass_list[j][2]) == im_UT+'.0': #If the times match, append the airmass
                header['AIRMASS'] = float(airmass_list[j][3])
                print 'Appending airmass '+str(airmass_list[j][3])+' to '+imlist[i], im[0].header['object']
                im.writeto(directory+imlist[i],overwrite=True)
        im.close()

In [ ]:
# example input
airmass_appender2('/Users/dahlek/Desktop/march2017/redo/calstars/','/Users/dahlek/Desktop/march2017/redo/calstars/star1','/Users/dahlek/Desktop/march2017/qvir')

In [ ]:
#programs to fit a Gaussian to the stars:
#From http://scipy-cookbook.readthedocs.io/items/FittingData.html

def gaussian(height, center_x, center_y, width_x, width_y):
    """Returns a gaussian function with the given parameters"""
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)

def moments(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution by calculating its
    moments """
    total = data.sum()
    X, Y = np.indices(data.shape)
    x = (X*data).sum()/total
    y = (Y*data).sum()/total
    col = data[:, int(y)]
    width_x = np.sqrt(np.abs((np.arange(col.size)-y)**2*col).sum()/col.sum())
    row = data[int(x), :]
    width_y = np.sqrt(np.abs((np.arange(row.size)-x)**2*row).sum()/row.sum())
    height = data.max()
    return height, x, y, width_x, width_y

def fitgaussian(data):
    """Returns (height, x, y, width_x, width_y)
    the gaussian parameters of a 2D distribution found by a fit"""
    params = moments(data)
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

def fwhm(width):
    """Takes widths from above programs, returns FWHM"""
    return 2*math.sqrt(2*np.log(2))*width

In [ ]:
# Issue with Gaussian fitting arises if there are bright pixels near border (result of low SNR issues). This routine elliminates bright pixels that aren't the star by looking at the border of the image and making any of the brightest pixels that are too close to the border 0.
# This routine might not be needed for different datasets; will be necessary if brightest pixels are in border region, since gaussian fitting routine looks for brightest pixel.
# Rerun until no bright pixels are found in the border area (will not print location of pixel if there are none in border)

directory = '/Users/dahlek/Desktop/march2017/redo/calstars/'
imlist = np.loadtxt(directory+'star8',dtype=str)

hot_images = []

for q in range(0,20): # Look in the border area 20 times. 
    print ' '
    for i in range(0,len(imlist)):
        im = fits.open(directory+imlist[i])
        border_size = im[0].header['NAXIS1'] # Assumes square data array
        if im[0].header['rfon'] == 1:
            #print imlist[i],  np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)
            # If the max value is within 50 pixels of the border:
            if np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0] > border_size-50 or np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1] > border_size-50 or np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0] < 50 or np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1] < 50:
                print imlist[i],im[0].header['lambda'],np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape),  ' is too close to the border'
                #hot_images.append(imlist[i])
                im[0].data[np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0]][np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1]] = 0
                print im[0].data[np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0]][np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1]]
                im.writeto(directory+imlist[i],overwrite = True)
        im.close()

In [ ]:
def ap_phot(directory,images,r_ap,r_annulus_inner,r_annulus_outer,filename):
    #directory = location of images
    #images = directory and name of list to look at
    #r_ap = apeture radius (in pixels; can code in arcsecond -> pixel conversion later)
    #r_annulus_inner and _outer = inner and outer radii of the bg annulus
    #If images have already been divided by exposure time: undo that before running them in this program. 
    #Header values will be in units of DN, or DN/sec, or other units that require the image to be in units of DN. 
    # Saves textfile of wl and flux of stars.
    
    #load imagelist:
    imagelist = np.loadtxt(images,dtype=str)
    
    sat_value = 30400 # 5 sigma value for new CCD
    #sat_value = 43836. #from linearity_tests.ipynb
    #readnoise = 10.4 #electrons - from alta f47 spec sheet. !!not counts?
    readnoise = 9.9/1.5 # 9.9 electrons - from spec sheet for new camera. Used dummy gain value of 1.5 e-/count
    bordersize = 1 #size of borders that will be ignored in annulus/apeture masks. border that was ignored in flats.
    
    exposure_time_query = 'y' # Since these images have been divided by exposure time, need to undo it briefly (will redo exposure time division at end of program)
    starflux = []
    wl = []
    
    print 'Doing photometry...'
    #Conduct photometry.
    for i in range(0,len(imagelist)):
        im = fits.open(directory+imagelist[i])
        if im[0].header['rfon'] == 1:
            
            if exposure_time_query == 'y': #if needs to be multiplied by exposure time to get image back in units of DN
                im[0].data *= im[0].header['exposure']
            
            print imagelist[i],im[0].header['lambda']
            
            #Fixing any nans, infs, or negative values that will fuck up gaussian fit
            im[0].data[im[0].data == np.nan] = 0
            im[0].data[im[0].data == np.inf] = 0
            #im[0].data[im[0].data < 0] = 0.01
            
            #define the center of the star by fitting a gaussian to a small region, ignorning the border of the image:
            gaussian_region_size = 30
            gauss = fitgaussian(abs(im[0].data[np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0]-gaussian_region_size:\
np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0]+gaussian_region_size,\
np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1]-gaussian_region_size:\
np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1]+gaussian_region_size]))
            
            x1, y1, fwhm_x, fwhm_y  = gauss[1], gauss[2], fwhm(gauss[3]), fwhm(gauss[4])
            
            # Redefine x1, y1 to be in the reference frame of the whole image
            x1 += np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[1]-gaussian_region_size
            y1 += np.unravel_index(np.argmax(im[0].data, axis=None), im[0].data.shape)[0]-gaussian_region_size
            
            x1 = int(round(x1)) # x-position of star
            y1 = int(round(y1)) # y-position of star
            
            # Once star has been located, make empty annulus and apeture masks:
            apeture = np.zeros((im[0].header['NAXIS1'],im[0].header['NAXIS2']))
            annulus = np.zeros((im[0].header['NAXIS1'],im[0].header['NAXIS2']))
            
            #make apeture and annulus mask
            for x in range(0,len(im[0].data)):
                for y in range(0,len(im[0].data[x])):
                    #qualifier: if x and y within radius:
                    if math.sqrt((x-x1)**2 + (y-y1)**2) <= r_ap:
                        apeture[y][x] = 1
                        
                    #qualifier: if x and y within radii:
                    if math.sqrt((x-x1)**2 + (y-y1)**2) <= r_annulus_outer\
                    and math.sqrt((x-x1)**2 + (y-y1)**2) >= r_annulus_inner: 
                        annulus[y][x] = 1
            
            #make un-corrected image borders 0 in masks; don't want to count borders that were not corrected for
            #otherwise, will see inf or nan and shit a brick
            #!assume array is square!
            for x in range(0,len(im[0].data)):
                for y in range(0,len(im[0].data[x])):
                    if x <= bordersize-1 or x >= len(im[0].data)-bordersize or\
                    y <= bordersize-1 or y >= len(im[0].data)-bordersize and annulus[x][y] == 1:
                        annulus[x][y] = 0.0
                    elif x <= bordersize-1 or x >= len(im[0].data)-bordersize or\
                    y <= bordersize-1 or y >= len(im[0].data)-bordersize and apeture[x][y] == 1:
                        apeture[x][y] = 0.0
            
            #plotting everything for reference
            #plt.matshow(im[0].data)
            #plt.matshow(apeture)
            #plt.plot(x1,y1,'ro')
            #plt.matshow(annulus)
            #plt.show()
            
            #find image values within apeture mask 
            n_ap = 0 #number of pixels within the apeture
            napersat = 0 #number of saturated pixels inside the aperture
            apeture_values = []
            
            #Record number of pixels, saturated pixels within aperture
            for x in range(0,len(im[0].data)):
                for y in range(0,len(im[0].data[x])):
                    if apeture[x][y] == 1:
                        if im[0].data[x][y] > 0: #Ignore negative values
                            apeture_values.append(im[0].data[x][y])
                            n_ap += 1
                            if im[0].data[x][y] >= sat_value:
                                napersat += 1

            print 'Number of pixels in aperture:',n_ap

            #print np.median(apeture_values)
            aperDN_total = sum(apeture_values) #total counts in aperture (DN)

            print 'Total DN in aperture (sum of all pixels within mask)',aperDN_total

            #collecting flux within the apeture: F_AP
            n_an = 0
            nskysat = 0
            annulus_values = []

            for x in range(0,len(im[0].data)):
                for y in range(0,len(im[0].data[x])):
                    if annulus[x][y] == 1:
                        if im[0].data[x][y] > 0: #Ignore negative values
                            annulus_values.append(im[0].data[x][y])
                            n_an += 1
                            if im[0].data[x][y] >= sat_value:
                                nskysat += 1


            print 'Number of pixels in annulus:',n_an

            F_BG = np.median(annulus_values) # median flux per pixel within background annulus
            skyDN = sum(annulus_values) # total counts in annulus

            print 'Median of all of the annulus values: (DN)',F_BG

            #total DN from star:
            starDN = aperDN_total-(n_ap*F_BG)
            print 'starDN, total DN in aperture - aperture size*median background DN',starDN
            flux = starDN/im[0].header['exposure'] #counts/sec
            print 'starDN/exposure time (star flux, dn/sec)',flux
            snr = starDN/math.sqrt(starDN+n_ap*(F_BG+readnoise)) #!!!!!!!!!!!!!readnoise = ??
            print 'Signal to noise',snr
            if snr < 0:
                print 'NEGATIVE SNR!!!'
                
            #plt.plot(annulus_values)
            #plt.plot(apeture_values)
            #plt.xlim(0,1000)
            #plt.show()
            
            print 'Median of annulus values', np.mean(annulus_values)
            
            starflux.append(flux)
            wl.append(im[0].header['lambda'])
            
            print ' '
            
            #attaching information to header (taking cues from Paul's ap_phot.pro)
            #header['keyword'] = (X,'comment')
            im[0].header['STAR_X'] = (x1,'star x-position')
            im[0].header['STAR_Y'] = (y1,'star y-position')
            im[0].header['STFWHMX'] = (fwhm_x,'star fwhm in x')
            im[0].header['STFWHMY'] = (fwhm_y,'star fwhm in y')
            im[0].header['APER_RAD'] = (r_ap,'radius (pixels) of star apeture')
            im[0].header['APERNPIX'] = (n_ap,'number of pixels in star apeture')
            im[0].header['APER_DN'] = (aperDN_total,'total DN in star aperture')
            im[0].header['APER_SAT'] = (napersat,'number of saturated pixels in star apeture')
            im[0].header['SKY_RAD1'] = (r_annulus_inner,'inner radius (pixels) of bg annulus')
            im[0].header['SKY_RAD2'] = (r_annulus_outer,'outer radius (pixels) of bg annulus')
            im[0].header['SKY_NPIX'] = (n_an,'number of pixels in sky aperture')
            im[0].header['SKY_DN'] = (skyDN,'total DN in sky annulus')
            im[0].header['SKY_SAT'] = (nskysat,'number of saturated pixels in sky annulus')
            im[0].header['BKG_DNPX'] = (F_BG,'median bg DN/pix') #***CHECK****
            im[0].header['STAR_DN'] = (starDN,'total DN from star [APER_DN-(APERNPIX*BKG_DNPX)]')
            im[0].header['STAR_FLX'] = (flux,'star flux (STAR_DN/EXPOSURE)')
            im[0].header['STAR_SNR'] = (snr,'star signal to noise ratio')
            
            #If multiplied by exposure time for this program, redo it so it doesn't get saved
            if exposure_time_query == 'y':
                im[0].data /= im[0].header['exposure']
            
            #Save image:
            im.writeto(directory+imagelist[i],overwrite=True)
            im.close()
            
    #write wavelength and flux to a file
    filearray = np.zeros((len(wl),2))
    filearray[:,0] = wl
    filearray[:,1] = starflux
    
    np.savetxt(filename,filearray)
    
    return wl,starflux

In [ ]:
# example input:
ap_phot('/Users/dahlek/Desktop/march2017/redo/calstars/','/Users/dahlek/Desktop/march2017/redo/calstars/star1',79,90,170,'/Users/dahlek/Desktop/march2017/redo/march2017_star1_flux')